# Introduction to Cloud Machine Learning with Flask API and CNTK



In [5]:
#load libraries
import os,sys
import pkg_resources
from flask import Flask, render_template, request, send_file

print("System version: {}".format(sys.version))
print("Flask version: {}".format(pkg_resources.get_distribution("flask").version))

System version: 2.7.12 |Anaconda 4.0.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]
Flask version: 0.10.1


In [6]:
app = Flask(__name__)